## Data -> Table

In [292]:
import pandas as pd
from itertools import groupby

In [530]:
descriptions =[]

with open("how_people_describe_themselves.txt","r")  as f:
    for i in f:
        descriptions.append(i.rstrip('\n'))

In [531]:
desc_array = [list(group) for k, group in groupby(descriptions, lambda x: x == " ") if not k]
    

In [295]:
# for i in desc_array:
#     i[1] = i[1].strip('Who you are: ')
#     i[2] = i[2].strip('What you are like: ')
#     i[3] = i[3].strip('What is the essence of what makes you YOU: ')

In [532]:
for i in desc_array:
    i[1] = i[1][13:]
    i[2] = i[2][19:]
    i[3] = i[3][42:]

In [533]:
df = pd.DataFrame(desc_array, columns=['person', 'who', 'likes', 'essence'])
df[:10]

,person,who,likes,essence
0,Person 1,I am a 28-year-old woman living in Massachusetts.,I'm neurotic and eccentric but good at heart.,I am obsessed with rabbit and love technology.
1,Person 2,I am dedicated and persistent and willing to g...,i'm fun loving and like to make people laugh b...,I am insightful and am able to fit together p...
2,Person 3,"I am a feminist, a student, and a homosexual.","I am kind, nurturing, caring, and genuine.","My honesty, my loyalty, and my generosity mak..."
3,Person 4,I'm an immigrant who became a citizen of the U...,"I can be stubborn and stand-offish, but I stil...","I'm introverted, so I need time alone to rech..."
4,Person 5,I'm a boring person,I'm quiet and introspective.,I have deep wells of sorrow.
5,Person 6,I am a girl who lives in New York City in my t...,I am very social and outgoing.,I am outgoing and care tremendously about my ...
6,Person 7,I am a mom of 3,"I like R &B Music, Japanese Food, Pizza",I am sympathetic.
7,Person 8,I am a licensed counselor.,I'm silly and funny.,I'm fun and sarcastic a lot of the time.
8,Person 9,"I'm Jim, a 53 year old american man","I'm forceful, happy, calm and occasionally gru...",I'm a drivan dominant at work and play. My wi...
9,Person 10,"I'm a fat, biracial bisexual woman.",I'm a funny depressive--aren't all depressives...,"I'm a boring drudge, not glamorous or fancy, ..."


## NLP

In [ ]:
# !pip install autocorrect

In [467]:
import nltk
from nltk.corpus import stopwords
from autocorrect import Speller

nltk.download('stopwords')
spell = Speller(lang='en')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akrishna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [579]:
def preprocess(text):
    """Pre-processes the text, splits into tokens that are lower-cased, filtered and lemmatized."""
    tokens = (t.lower() for t in nltk.word_tokenize(text)
                            if t.isalpha()
                            and t.lower() not in stopwords.words())

    # wordnet_lemmatizer = nltk.WordNetLemmatizer()
    return [spell(t) for t in tokens]

In [580]:
pos_tagged_desc = []
for i in desc_array:
    x = [nltk.pos_tag(preprocess(i[1])), nltk.pos_tag(preprocess(i[2])), nltk.pos_tag(preprocess(i[3]))]
    pos_tagged_desc.append(x)

In [581]:
pos_tagged_desc[1]

[[('dedicated', 'VBN'),
  ('persistent', 'JJ'),
  ('willing', 'JJ'),
  ('go', 'VB'),
  ('extra', 'JJ'),
  ('mile', 'NN')],
 [('fun', 'NN'),
  ('loving', 'VBG'),
  ('like', 'IN'),
  ('make', 'NN'),
  ('people', 'NNS'),
  ('laugh', 'IN'),
  ('telling', 'VBG'),
  ('jokes', 'NNS'),
  ('making', 'VBG'),
  ('light', 'JJ'),
  ('situations', 'NNS')],
 [('insightful', 'JJ'),
  ('able', 'JJ'),
  ('fit', 'NN'),
  ('together', 'RB'),
  ('pieces', 'NNS'),
  ('understand', 'VBP'),
  ('big', 'JJ'),
  ('picture', 'NN')]]

In [582]:
descriptors = []
for i in pos_tagged_desc:
    s = ""
    for j in i:
        for k in j:
            s += k[0]+" "
    descriptors.append(s)

In [583]:
descriptors[:10]

['woman living massachusetts neurotic eccentric good heart obsessed rabbit love technology ',
 'dedicated persistent willing go extra mile fun loving like make people laugh telling jokes making light situations insightful able fit together pieces understand big picture ',
 'feminist student homosexual kind nurturing caring genuine honesty loyalty generosity make ',
 'immigrant became citizen united states american stubborn still much friends family nerd love video games prefer stay home introverted need time alone recharge recalibrate ',
 'boring person quiet introspective deep wells sorrow ',
 'girl lives new york city twenties incredibly close family social outgoing outgoing tremendously friends ',
 'mom like r b music japanese food pizza sympathetic ',
 'licensed counselor silly funny fun sarcastic lot time ',
 'year old american forceful happy calm occasionally grumpy touch sarcasm driven dominant work play wife essential life lost without grounding ',
 'fat biracial bisexual woman

## TF-IDF

In [473]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### For each description as a document

In [539]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(descriptors)

df_idf = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names(),columns=["idf_weights"]).sort_values('idf_weights', ascending=False)[:20]
df_idf.sort_values(by=['idf_weights'], ascending=False)[:10]

,idf_weights
eccentric,0.227588
living massachusetts,0.227588
neurotic,0.227588
heart obsessed,0.227588
rabbit,0.227588
neurotic eccentric,0.227588
technology,0.227588
rabbit love,0.227588
living,0.227588
good heart,0.227588


#### For each of the questions as a document

In [584]:
who = []
likes = []
essence = []
for i in range(len(pos_tagged_desc)):
        who.append(pos_tagged_desc[i][0])
        likes.append(pos_tagged_desc[i][1])
        essence.append(pos_tagged_desc[i][2])

In [585]:
who_list = []
for i in who:
    s = ""
    for j in i:
            s += j[0]+" "
    who_list.append(s)
    
likes_list = []
for i in likes:
    s = ""
    for j in i:
            s += j[0]+" "
    likes_list.append(s)
    
essence_list = []
for i in essence:
    s = ""
    for j in i:
            s += j[0]+" "
    essence_list.append(s)

In [586]:
who_vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_who = who_vectorizer.fit_transform(who_list)

likes_vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_likes = likes_vectorizer.fit_transform(likes_list)

essence_vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_essence = essence_vectorizer.fit_transform(essence_list)

In [587]:
def who_top(i):
    df_idf = pd.DataFrame(X_who[i].T.todense(), index=who_vectorizer.get_feature_names(),columns=["idf_weights"])
    return df_idf.sort_values(by=['idf_weights'], ascending=False).index[:1].tolist()

In [588]:
def likes_top(i):
    df_idf = pd.DataFrame(X_likes[i].T.todense(), index=likes_vectorizer.get_feature_names(),columns=["idf_weights"])
    return df_idf.sort_values(by=['idf_weights'], ascending=False).index[:1].tolist()

In [589]:
def essence_top(i):
    df_idf = pd.DataFrame(X_essence[i].T.todense(), index=essence_vectorizer.get_feature_names(),columns=["idf_weights"])
    return df_idf.sort_values(by=['idf_weights'], ascending=False).index[:1].tolist()

## RAKE

In [ ]:
# !pip install rake-nltk

In [ ]:
from rake_nltk import Rake, Metric

r = Rake(ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO)
r.extract_keywords_from_text(desc_array[0][1])
r.get_ranked_phrases()

## Top-3

#### For each description as a document

In [546]:
top3_all = []
for i in X:
    top3_desc = pd.DataFrame(i.T.todense(), index=vectorizer.get_feature_names(),columns=["idf_weights"]).sort_values('idf_weights', ascending=False)
    top3_all.append(top3_desc.index[:3].tolist())

In [547]:
top3_all[:10]

[['eccentric', 'living', 'eccentric good'],
 ['light', 'pieces understand', 'picture'],
 ['generosity make', 'homosexual', 'caring genuine'],
 ['home', 'citizen', 'family nerd'],
 ['wells sorrow', 'sorrow', 'deep wells'],
 ['outgoing', 'new york', 'girl lives'],
 ['sympathetic', 'pizza', 'food pizza'],
 ['licensed', 'fun sarcastic', 'sarcastic'],
 ['sarcasm driven', 'lost', 'play wife'],
 ['biracial bisexual', 'fat biracial', 'fairly dependable']]

#### For each of the questions as a document

In [590]:
top3_each = []
for i in range(50):
    temp = []
    temp.append(who_top(i)[0])
    temp.append(likes_top(i)[0])
    temp.append(essence_top(i)[0])
    top3_each.append(temp)

In [591]:
top3_each[:10]

[['living massachusetts', 'eccentric good', 'rabbit'],
 ['willing', 'people', 'picture'],
 ['feminist', 'genuine', 'loyalty generosity'],
 ['immigrant became', 'prefer', 'alone'],
 ['boring', 'quiet introspective', 'wells sorrow'],
 ['city twenties', 'social outgoing', 'tremendously friends'],
 ['mom', 'music', 'sympathetic'],
 ['licensed counselor', 'silly funny', 'sarcastic lot'],
 ['old american', 'occasionally grumpy', 'play'],
 ['fat biracial', 'riot', 'drudge glamorous']]

## NounProject

In [284]:
import requests, json
from requests_oauthlib import OAuth1
from PIL import Image
import urllib.request
import os

In [562]:
png_dir = 'docs/NounProjectOutputs/png_images/'
jpg_dir = 'docs/NounProjectOutputs/jpg_images/'

In [568]:
auth = OAuth1("1aac7c276b39401f9f042c53e8f8e5d6", "38b8f9bf4a074812918528889a076fa1")

not_found = []
no_icon_url = []

l = os.listdir(png_dir)

for search_terms in top3_each:
    for search_term in search_terms:    
        if ' ' in search_term:
            search_term = search_term.split(' ')[-1]
            
        if search_term + '.png' in l:
            continue
        
        endpoint = "http://api.thenounproject.com/icon/" + search_term
        response = requests.get(endpoint, auth=auth)
        
        if(response.status_code==404):
            not_found.append(search_term)
            continue
            
        parsed_response = response.json()
        
        if 'preview_url' not in parsed_response['icon']:
            no_icon_url.append(search_term)
            continue
            
        icon_url = parsed_response['icon']['preview_url']
        urllib.request.urlretrieve(icon_url, png_dir + search_term + ".png")

In [569]:
not_found

['unbrainwashed', 'johnson', 'principals']

In [329]:
not_found_all

In [281]:
not_found_each

['laidback', 'henry', 'deeply']

In [563]:
def png2jpg(image_path):
    image = Image.open('docs/NounProjectOutputs/png_images/' + image_path).convert("RGBA")
    new_image = Image.new("RGBA", image.size, "WHITE")
    new_image.paste(image, (0, 0), image)
    new_image.convert('RGB').save(jpg_dir + image_path.split('.')[0] + '.jpg', "JPEG") 

In [564]:
for pngFile in os.listdir('docs/NounProjectOutputs/png_images/'):
    png2jpg(pngFile)

## Layout

In [554]:
import math
from PIL import Image

def arrangeImagesInCircle(masterImage, imagesToArrange, radius):
    imgWidth, imgHeight = masterImage.size

#     diameter = min(
#         imgWidth  - max(img.size[0] for img in imagesToArrange),
#         imgHeight - max(img.size[1] for img in imagesToArrange)
#     )
#     radius = diameter / 2

    circleCenterX = imgWidth  / 2
    circleCenterY = imgHeight / 2
    theta = 2*math.pi / len(imagesToArrange)
    for i, curImg in enumerate(imagesToArrange):
        angle = i * theta
        dx = int(radius * math.cos(angle))
        dy = int(radius * math.sin(angle))

        pos = (
            int(circleCenterX + dx - curImg.size[0]/2),
            int(circleCenterY + dy - curImg.size[1]/2)
        )
        masterImage.paste(curImg, pos, mask=curImg)

In [570]:
pngFilenames = []
for search_terms in top3_each:
    pngTemp = []
    for search_term in search_terms:
        if ' ' in search_term:
            search_term = search_term.split(' ')[-1]
        if search_term not in not_found:
            pngTemp.append(png_dir + search_term + '.png')
    pngFilenames.append(pngTemp)

In [571]:
pngFilenames

[['docs/NounProjectOutputs/png_images/massachusetts.png',
  'docs/NounProjectOutputs/png_images/good.png',
  'docs/NounProjectOutputs/png_images/rabbit.png'],
 ['docs/NounProjectOutputs/png_images/willing.png',
  'docs/NounProjectOutputs/png_images/people.png',
  'docs/NounProjectOutputs/png_images/picture.png'],
 ['docs/NounProjectOutputs/png_images/feminist.png',
  'docs/NounProjectOutputs/png_images/genuine.png',
  'docs/NounProjectOutputs/png_images/generosity.png'],
 ['docs/NounProjectOutputs/png_images/became.png',
  'docs/NounProjectOutputs/png_images/prefer.png',
  'docs/NounProjectOutputs/png_images/alone.png'],
 ['docs/NounProjectOutputs/png_images/boring.png',
  'docs/NounProjectOutputs/png_images/introspective.png',
  'docs/NounProjectOutputs/png_images/sorrow.png'],
 ['docs/NounProjectOutputs/png_images/twenties.png',
  'docs/NounProjectOutputs/png_images/outgoing.png',
  'docs/NounProjectOutputs/png_images/friends.png'],
 ['docs/NounProjectOutputs/png_images/mom.png',
  '

In [ ]:
jpgFilenames = [[jpg_dir + search_term + '.jpg' for search_term in search_terms if search_term not in not_found] for search_terms in top3_each]

In [572]:
for i in range(50):
    img = Image.new("RGB", (600,600), "WHITE")

    images = [Image.open(filename) for filename in pngFilenames[i]]
    arrangeImagesInCircle(img, images, 100)

    img.convert('RGB').save("docs/EssenceOutputs-ngrams/essence-" + str(i) + ".jpg", "JPEG")

## Detect Mood

In [25]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [503]:
data = pd.read_csv('text_emotion.csv')

# sentiment_list = ['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness', 
#                   'hate', 'love', 'neutral', 'relief', 'sadness', 'surprise', 'worry']

data = data.drop('author', axis=1)
data = data.drop(data[data.sentiment == 'boredom'].index)
data = data.drop(data[data.sentiment == 'empty'].index)
data = data.drop(data[data.sentiment == 'fun'].index)
data = data.drop(data[data.sentiment == 'relief'].index)
data = data.drop(data[data.sentiment == 'surprise'].index)
data = data.drop(data[data.sentiment == 'sadness'].index)
data = data.drop(data[data.sentiment == 'fun'].index)
data = data.drop(data[data.sentiment == 'enthusiasm'].index)

# Making all letters lowercase
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# Removing Punctuation, Symbols
data['content'] = data['content'].str.replace('[^\w\s]',' ')

# Removing Stop Words using NLTK
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# #Lemmatisation
# data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
#Correcting Letter Repetitions

def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

data['content'] = data['content'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))

# Code to find the top 10,000 rarest words appearing in the data
freq = pd.Series(' '.join(data['content']).split()).value_counts()[-10000:]

# Removing all those rarely appearing words from the data
freq = list(freq.index)
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

#Encoding output labels 'sadness' as '1' & 'happiness' as '0'
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.sentiment.values)

# Splitting into training and testing data in 90:10 ratio
X_train, X_val, y_train, y_val = train_test_split(data.content.values, y, stratify=y, random_state=42, test_size=0.1, shuffle=True)

# Extracting TF-IDF parameters
# tfidf = TfidfVectorizer(max_features=1000, analyzer='word',ngram_range=(1,3))
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_val_tfidf = tfidf.fit_transform(X_val)

# Extracting Count Vectors Parameters
count_vect = CountVectorizer(analyzer='word')
count_vect.fit(data['content'])
X_train_count =  count_vect.transform(X_train)
X_val_count =  count_vect.transform(X_val)

In [504]:
list(enumerate(lbl_enc.classes_))

[(0, 'anger'),
 (1, 'happiness'),
 (2, 'hate'),
 (3, 'love'),
 (4, 'neutral'),
 (5, 'worry')]

In [505]:
# Model 1: Linear SVM
lsvm = SGDClassifier(alpha=0.001, random_state=5, max_iter=15, tol=None)
lsvm.fit(X_train_count, y_train)
y_pred = lsvm.predict(X_val_count)
print('lsvm using count vectors accuracy %s' % accuracy_score(y_pred, y_val))

# Model 2: Logistic Regression
logreg = LogisticRegression(C=1)
logreg.fit(X_train_count, y_train)
y_pred = logreg.predict(X_val_count)
print('log reg count vectors accuracy %s' % accuracy_score(y_pred, y_val))

lsvm using count vectors accuracy 0.5070677781805002


//anaconda3/envs/cs231n/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/envs/cs231n/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


log reg count vectors accuracy 0.501268575570859


In [506]:
desc_count_vectors = count_vect.transform(descriptors)

desc_emotion_pred = lsvm.predict(desc_count_vectors)
print(desc_emotion_pred)

[3 3 4 3 5 5 4 1 5 3 3 4 3 4 4 1 3 5 5 5 3 1 1 1 3 5 5 3 5 3 3 3 3 5 1 5 3
 3 3 3 3 3 1 3 5 3 4 3 3 1]


In [253]:
from IPython.display import Image, display, HTML

In [507]:
desc_emotions = []
k = 0
for i in desc_array:
    desc_emotions.append((i, lbl_enc.classes_[desc_emotion_pred[k]]))
    k += 1

In [508]:
desc_emotions

[(['Person 1',
   'I am a 28-year-old woman living in Massachusetts.',
   "I'm neurotic and eccentric but good at heart.",
   'I am obsessed with rabbits and love technology.'],
  'love'),
 (['Person 2',
   'I am dedicated and persistent and willing to go the extra mil',
   "'m fun loving and like to make people laugh by telling jokes and making light of situations",
   'I am insightful and am able to fit together pieces to understand the big pictur'],
  'love'),
 (['Person 3',
   'I am a feminist, a student, and a homosexual.',
   'I am kind, nurturing, caring, and genuine.',
   'My honesty, my loyalty, and my generosity make me me.'],
  'neutral'),
 (['Person 4',
   "I'm an immigrant who became a citizen of the United States. I'm an American.",
   "I can be stubborn and stand-offish, but I still care very much about my friends and family. I'm a nerd about movies/tvshows/books. I love video games and prefer to stay home.",
   "I'm introverted, so I need time alone to recharge and reca

## Essence - All

In [260]:
# for i in range(50):
#     print(desc_array[i])
#     print(top3_all[i], '\n')
#     image1 = jpg_dir + top3_all[i][0] + '.jpg'
#     image2 = jpg_dir + top3_all[i][1] + '.jpg'
#     image3 = jpg_dir + top3_all[i][2] + '.jpg'
#     output = 'docs/EssenceOutputs/essence-' + str(i) + '.jpg'
#     display(HTML("<table><tr><td><img src='"+ image1 + "'></td><td><img src='" + image2 + "'></td><td><img src='"+ image3 + "'></td><td></td><td><img src='"+ output + "'></td></tr></table>"))

['Person 1', 'I am a 28-year-old woman living in Massachusetts.', "I'm neurotic and eccentric but good at heart.", 'I am obsessed with rabbits and love technology.']
['living', 'obsessed', 'rabbits'] 



,,,,


['Person 2', 'I am dedicated and persistent and willing to go the extra mile', "i'm fun loving and like to make people laugh by telling jokes and making light of situations", 'I am insightful and am able to fit together pieces to understand the big picture']
['mile', 'together', 'situations'] 



,,,,


['Person 3', 'I am a feminist, a student, and a homosexual.', 'I am kind, nurturing, caring, and genuine.', 'My honesty, my loyalty, and my generosity make me me.']
['generosity', 'loyalty', 'homosexual'] 



...

,,,,


['Person 49', 'I am a working mom.', 'I am calm, determined, and loving.', 'The essence of what makes me is my compassion and caring.']
['compassion', 'determined', 'calm'] 



,,,,


['Person 50', 'I am a 27 year old special education teacher who lives in California.', 'I am a sociable and very goofy young guy who likes to make everyone laugh.', 'The essence of what makes me myself is that I am always generally positive and like to spread a smile to all those around me!']
['goofy', 'generally', 'positive'] 



,,,,


## Essence - Individual

In [288]:
# for i in range(50):
#     print(desc_array[i])
#     print(top3_each[i], '\n')
#     image1 = jpg_dir + top3_each[i][0] + '.jpg'
#     image2 = jpg_dir + top3_each[i][1] + '.jpg'
#     image3 = jpg_dir + top3_each[i][2] + '.jpg'
#     output = 'docs/EssenceOutputs-individual/essence-' + str(i) + '.jpg'
#     display(HTML("<table><tr><td><img src='"+ image1 + "'></td><td><img src='" + image2 + "'></td><td><img src='"+ image3 + "'></td><td></td><td><img src='"+ output + "'></td></tr></table>"))

['Person 1', 'I am a 28-year-old woman living in Massachusetts.', "I'm neurotic and eccentric but good at heart.", 'I am obsessed with rabbits and love technology.']
['living', 'neurotic', 'obsessed'] 



,,,,


['Person 2', 'I am dedicated and persistent and willing to go the extra mile', "i'm fun loving and like to make people laugh by telling jokes and making light of situations", 'I am insightful and am able to fit together pieces to understand the big picture']
['extra', 'light', 'understand'] 



,,,,


['Person 3', 'I am a feminist, a student, and a homosexual.', 'I am kind, nurturing, caring, and genuine.', 'My honesty, my loyalty, and my generosity make me me.']
['homosexual', 'genuine', 'loyalty'] 



...

,,,,


['Person 49', 'I am a working mom.', 'I am calm, determined, and loving.', 'The essence of what makes me is my compassion and caring.']
['working', 'calm', 'compassion'] 



,,,,


['Person 50', 'I am a 27 year old special education teacher who lives in California.', 'I am a sociable and very goofy young guy who likes to make everyone laugh.', 'The essence of what makes me myself is that I am always generally positive and like to spread a smile to all those around me!']
['education', 'young', 'generally'] 



,,,,


## Old Essence

In [332]:
for i in range(50):
    print(desc_array[i])
    print(top3_all[i], '\n')
    image1 = jpg_dir + top3_all[i][0] + '.jpg'
    image2 = jpg_dir + top3_all[i][1] + '.jpg'
    image3 = jpg_dir + top3_all[i][2] + '.jpg'
    output = 'docs/OldEssenceOutputs/essence-' + str(i) + '.jpg'
    display(HTML("<table><tr><td><img src='"+ image1 + "'></td><td><img src='" + image2 + "'></td><td><img src='"+ image3 + "'></td><td></td><td><img src='"+ output + "'></td></tr></table>"))

['Person 1', 'I am a 28-year-old woman living in Massachusetts.', "I'm neurotic and eccentric but good at heart.", 'I am obsessed with rabbits and love technology.']
['technology', 'obsessed', 'massachusetts'] 



,,,,


['Person 2', 'I am dedicated and persistent and willing to go the extra mil', "'m fun loving and like to make people laugh by telling jokes and making light of situations", 'I am insightful and am able to fit together pieces to understand the big pictur']
['jokes', 'mil', 'dedicated'] 



,,,,


['Person 3', 'I am a feminist, a student, and a homosexual.', 'I am kind, nurturing, caring, and genuine.', 'My honesty, my loyalty, and my generosity make me me.']
['feminist', 'generosity', 'homosexual'] 



...

,,,,


['Person 49', 'I am a working mom.', 'I am calm, determined, and loving.', 'The essence of what makes me is my compassion and caring.']
['compassion', 'determined', 'calm'] 



,,,,


['Person 50', 'I am a 27 year old special education teacher who lives in California.', 'I am a sociable and very goofy young guy who likes to make everyone laugh.', 'The essence of what makes me myself is that I am always generally positive and like to spread a smile to all those around me!']
['goofy', 'generally', 'spread'] 



,,,,


## Essence - ngrams

In [574]:
for i in range(50):
    print(desc_array[i])
    print(top3_each[i], '\n')
    if ' ' in top3_each[i][0]:
        image1 = jpg_dir + top3_each[i][0].split(' ')[-1] + '.jpg'
    else:
        image1 = jpg_dir + top3_each[i][0] + '.jpg'
    if ' ' in top3_each[i][1]:
        image2 = jpg_dir + top3_each[i][1].split(' ')[-1] + '.jpg'
    else:
        image2 = jpg_dir + top3_each[i][1] + '.jpg'
    if ' ' in top3_each[i][2]:
        image3 = jpg_dir + top3_each[i][2].split(' ')[-1] + '.jpg'
    else:
        image3 = jpg_dir + top3_each[i][2] + '.jpg'
    output = 'docs/EssenceOutputs-ngrams/essence-' + str(i) + '.jpg'
    display(HTML("<table><tr><td><img src='"+ image1 + "'></td><td><img src='" + image2 + "'></td><td><img src='"+ image3 + "'></td><td></td><td><img src='"+ output + "'></td></tr></table>"))

['Person 1', 'I am a 28-year-old woman living in Massachusetts.', "I'm neurotic and eccentric but good at heart.", ' I am obsessed with rabbit and love technology.']
['living massachusetts', 'eccentric good', 'rabbit'] 



,,,,


['Person 2', 'I am dedicated and persistent and willing to go the extra mile', "i'm fun loving and like to make people laugh by telling jokes and making light of situations", ' I am insightful and am able to fit together pieces to understand the big picture']
['willing', 'people', 'picture'] 



,,,,


['Person 3', 'I am a feminist, a student, and a homosexual.', 'I am kind, nurturing, caring, and genuine.', ' My honesty, my loyalty, and my generosity make me me.']
['feminist', 'genuine', 'loyalty generosity'] 



...

,,,,


['Person 49', 'I am a working mom.', 'I am calm, determined, and loving.', ' The essence of what makes me is my compassion and caring.']
['working mom', 'determined loving', 'compassion caring'] 



,,,,


['Person 50', 'I am a 27 year old special education teacher who lives in California.', 'I am a sociable and very goofy young guy who likes to make everyone laugh.', ' The essence of what makes me myself is that I am always generally positive and like to spread a smile to all those around me!']
['education', 'young guy', 'generally'] 



,,,,


## Good Essence Outputs

In [597]:
for i in os.listdir('docs/FinalOutputs/essence'):
    print(i)
    image1 = 'docs/FinalOutputs/essence/' + i
#     number = int(i.split('.')[0].split('-')[-1])
#     print(top3_each[number])
    display(HTML("<table><tr><td><img src='"+ image1 + "'></td></tr></table>"))

essence-10.jpg


""


essence-38.jpg


""


essence-4.jpg


...

""


essence-33.jpg


""


essence-22.jpg


""


## Style Outputs

In [595]:
for i in os.listdir('docs/FinalOutputs/essence-style'):
    image1 = 'docs/FinalOutputs/essence-style/' + i
#     number = int(i.split('.')[0].split('-')[-1])
#     print(top3_each[number])
    display(HTML("<table><tr><td><img src='"+ image1 + "'></td></tr></table>"))

""


""


""


""


""


...

""


""


""


""


""
